In [1]:
# Install required packages
!pip install transformers torch datasets evaluate sacrebleu rouge-score scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.4 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=19af56ed7309e64723c989683a252d0979cd1e2f2e51958c9d4202911cea41fb
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [2]:
import torch
from transformers import (
    pipeline, AutoTokenizer, AutoModel, AutoModelForMaskedLM,
    AutoModelForTokenClassification, TrainingArguments, Trainer,
    AutoModelForSequenceClassification, DataCollatorWithPadding
)
from datasets import Dataset, DatasetDict
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import accuracy_score
import pandas as pd

print("All packages installed and imported successfully!")
print("="*60)

# =============================================================================
# TASK 1: SENTIMENT ANALYSIS (One Line)
# =============================================================================
print("TASK 1: SENTIMENT ANALYSIS")
print("-" * 30)

# Initialize sentiment classifier
sentiment_classifier = pipeline("sentiment-analysis",
                               model="cardiffnlp/twitter-roberta-base-sentiment-latest")

# Test sentences
test_sentences = [
    "I love this movie, it's amazing!",
    "This product is terrible and broken.",
    "The weather is okay today."
]

print("Testing sentiment analysis:")
for sentence in test_sentences:
    result = sentiment_classifier(sentence)
    print(f"Text: '{sentence}'")
    print(f"Sentiment: {result[0]['label']} (confidence: {result[0]['score']:.3f})")
    print()

print("Reflection: The pretrained RoBERTa model effectively classifies sentiment with high confidence scores.")
print("="*60)


All packages installed and imported successfully!
TASK 1: SENTIMENT ANALYSIS
------------------------------


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/929 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cpu


Testing sentiment analysis:
Text: 'I love this movie, it's amazing!'
Sentiment: positive (confidence: 0.987)

Text: 'This product is terrible and broken.'
Sentiment: negative (confidence: 0.946)

Text: 'The weather is okay today.'
Sentiment: positive (confidence: 0.946)

Reflection: The pretrained RoBERTa model effectively classifies sentiment with high confidence scores.


In [3]:
# =============================================================================
# TASK 2: ZERO-SHOT TOPIC TAGGING
# =============================================================================
print("TASK 2: ZERO-SHOT TOPIC TAGGING")
print("-" * 30)

# Initialize zero-shot classifier
zero_shot_classifier = pipeline("zero-shot-classification",
                                model="facebook/bart-large-mnli")

# Define topics and test text
topics = ["sports", "technology", "politics", "entertainment", "health"]
test_texts = [
    "The new iPhone 15 features an improved camera system with better low-light performance.",
    "The football match was exciting with a last-minute goal deciding the winner.",
    "The president announced new policies regarding healthcare reform."
]

print("Testing zero-shot topic classification:")
for text in test_texts:
    result = zero_shot_classifier(text, topics)
    print(f"Text: '{text}'")
    print(f"Top topic: {result['labels'][0]} (score: {result['scores'][0]:.3f})")
    print(f"All scores: {dict(zip(result['labels'], [round(s, 3) for s in result['scores']]))}")
    print()

print("Reflection: Zero-shot classification works well without training, using natural language inference.")
print("="*60)


TASK 2: ZERO-SHOT TOPIC TAGGING
------------------------------


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


Testing zero-shot topic classification:
Text: 'The new iPhone 15 features an improved camera system with better low-light performance.'
Top topic: technology (score: 0.958)
All scores: {'technology': 0.958, 'entertainment': 0.019, 'health': 0.012, 'sports': 0.006, 'politics': 0.005}

Text: 'The football match was exciting with a last-minute goal deciding the winner.'
Top topic: sports (score: 0.632)
All scores: {'sports': 0.632, 'entertainment': 0.363, 'technology': 0.003, 'health': 0.001, 'politics': 0.001}

Text: 'The president announced new policies regarding healthcare reform.'
Top topic: health (score: 0.899)
All scores: {'health': 0.899, 'politics': 0.092, 'technology': 0.004, 'sports': 0.003, 'entertainment': 0.003}

Reflection: Zero-shot classification works well without training, using natural language inference.


In [4]:
# =============================================================================
# TASK 3: ENGLISH ↔ HINDI TRANSLATION
# =============================================================================
print("TASK 3: ENGLISH ↔ HINDI TRANSLATION")
print("-" * 30)

# Initialize translators
en_to_hi_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-en-hi")
hi_to_en_translator = pipeline("translation", model="Helsinki-NLP/opus-mt-hi-en")

# Test translations
en_text = "Hello, how are you today?"
hi_text = "आप कैसे हैं?"

print("English to Hindi:")
en_to_hi_result = en_to_hi_translator(en_text)
print(f"English: {en_text}")
print(f"Hindi: {en_to_hi_result[0]['translation_text']}")
print()

print("Hindi to English:")
hi_to_en_result = hi_to_en_translator(hi_text)
print(f"Hindi: {hi_text}")
print(f"English: {hi_to_en_result[0]['translation_text']}")

print("\nReflection: Machine translation works both ways, though quality depends on training data availability.")
print("="*60)


TASK 3: ENGLISH ↔ HINDI TRANSLATION
------------------------------


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/306M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/306M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

...

source.spm:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/813k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

Device set to use cpu


English to Hindi:
English: Hello, how are you today?
Hindi: हैलो, आप आज कैसे कर रहे हैं?

Hindi to English:
Hindi: आप कैसे हैं?
English: How are you?

Reflection: Machine translation works both ways, though quality depends on training data availability.


In [6]:
# =============================================================================
# TASK 4: ONE-PARAGRAPH SUMMARIZER
# =============================================================================
print("TASK 4: ONE-PARAGRAPH SUMMARIZER")
print("-" * 30)

# Initialize summarizer
summarizer = pipeline("summarization", model="t5-small")

# Test article
article = """
Artificial intelligence has made tremendous progress in recent years, particularly in the field of natural language processing.
Large language models like GPT-3 and GPT-4 have demonstrated remarkable capabilities in understanding and generating human-like text.
These models are trained on vast amounts of text data and can perform a wide variety of tasks, from answering questions to writing
creative content. However, there are still challenges to overcome, including issues with bias, factual accuracy, and computational
requirements. Researchers continue to work on improving these systems while also addressing ethical concerns about their deployment
and potential impact on society. The future of AI looks promising, with applications spanning education, healthcare, entertainment,
and many other domains.
"""

print("Original article:")
print(article.strip())
print("\nSummary:")
summary = summarizer(article, max_length=50, min_length=20, do_sample=False)
print(summary[0]['summary_text'])

print("\nReflection: T5-small effectively condenses the main points while maintaining key information.")
print("="*60)


TASK 4: ONE-PARAGRAPH SUMMARIZER
------------------------------


Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=50) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Original article:
Artificial intelligence has made tremendous progress in recent years, particularly in the field of natural language processing. 
Large language models like GPT-3 and GPT-4 have demonstrated remarkable capabilities in understanding and generating human-like text. 
These models are trained on vast amounts of text data and can perform a wide variety of tasks, from answering questions to writing 
creative content. However, there are still challenges to overcome, including issues with bias, factual accuracy, and computational 
requirements. Researchers continue to work on improving these systems while also addressing ethical concerns about their deployment 
and potential impact on society. The future of AI looks promising, with applications spanning education, healthcare, entertainment, 
and many other domains.

Summary:
large language models like GPT-3 and GPT-4 have demonstrated remarkable capabilities in understanding and generating human-like text . these models are tr

In [7]:
# =============================================================================
# TASK 5: EXTRACTIVE QA (Answer from passage)
# =============================================================================
print("TASK 5: EXTRACTIVE QA")
print("-" * 30)

# Initialize QA pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-cased-distilled-squad")

# Test context and questions
context = """
The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
It is named after the engineer Gustave Eiffel, whose company designed and built the tower.
Constructed from 1887 to 1889, it was the world's tallest structure until 1930.
The tower is 324 meters tall, approximately the same height as an 81-story building.
"""

questions = [
    "Who is the Eiffel Tower named after?",
    "How tall is the Eiffel Tower?",
    "When was the Eiffel Tower constructed?"
]

print("Context:", context.strip())
print("\nQuestions and Answers:")
for question in questions:
    result = qa_pipeline(question=question, context=context)
    print(f"Q: {question}")
    print(f"A: {result['answer']} (confidence: {result['score']:.3f})")
    print()

print("Reflection: Extractive QA successfully finds specific answers within the given context.")
print("="*60)


TASK 5: EXTRACTIVE QA
------------------------------


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/261M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

Device set to use cpu


Context: The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France. 
It is named after the engineer Gustave Eiffel, whose company designed and built the tower. 
Constructed from 1887 to 1889, it was the world's tallest structure until 1930. 
The tower is 324 meters tall, approximately the same height as an 81-story building.

Questions and Answers:
Q: Who is the Eiffel Tower named after?
A: Gustave Eiffel (confidence: 0.957)

Q: How tall is the Eiffel Tower?
A: 324 meters (confidence: 0.661)

Q: When was the Eiffel Tower constructed?
A: 1930 (confidence: 0.418)

Reflection: Extractive QA successfully finds specific answers within the given context.


In [8]:
# =============================================================================
# TASK 6: FILL-THE-BLANK WITH BERT
# =============================================================================
print("TASK 6: FILL-THE-BLANK WITH BERT")
print("-" * 30)

# Initialize fill-mask pipeline
fill_mask = pipeline("fill-mask", model="bert-base-uncased")

# Test sentences with masks
test_sentences = [
    "The capital of France is [MASK].",
    "I love to eat [MASK] for breakfast.",
    "The [MASK] is shining brightly today."
]

print("Testing masked token predictions:")
for sentence in test_sentences:
    results = fill_mask(sentence)
    print(f"Sentence: {sentence}")
    print("Top 3 predictions:")
    for i, result in enumerate(results[:3], 1):
        print(f"  {i}. {result['token_str']} (score: {result['score']:.3f})")
    print()

print("Reflection: BERT's masked language modeling shows contextual understanding by predicting appropriate words.")
print("="*60)


TASK 6: FILL-THE-BLANK WITH BERT
------------------------------


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Device set to use cpu


Testing masked token predictions:
Sentence: The capital of France is [MASK].
Top 3 predictions:
  1. paris (score: 0.417)
  2. lille (score: 0.071)
  3. lyon (score: 0.063)

Sentence: I love to eat [MASK] for breakfast.
Top 3 predictions:
  1. it (score: 0.135)
  2. them (score: 0.086)
  3. you (score: 0.072)

Sentence: The [MASK] is shining brightly today.
Top 3 predictions:
  1. sun (score: 0.740)
  2. moon (score: 0.029)
  3. city (score: 0.023)

Reflection: BERT's masked language modeling shows contextual understanding by predicting appropriate words.


In [9]:
# =============================================================================
# TASK 7: NAMED ENTITY RECOGNITION (NER)
# =============================================================================
print("TASK 7: NAMED ENTITY RECOGNITION")
print("-" * 30)

# Initialize NER pipeline
ner_pipeline = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english",
                       aggregation_strategy="simple")

# Test text
text = "Apple Inc. was founded by Steve Jobs in Cupertino, California. The company is now led by Tim Cook."

print(f"Text: {text}")
print("\nNamed Entities:")
entities = ner_pipeline(text)
for entity in entities:
    print(f"Entity: '{entity['word']}' | Type: {entity['entity_group']} | Confidence: {entity['score']:.3f}")

print("\nReflection: NER successfully identifies people, organizations, and locations with high accuracy.")
print("="*60)


TASK 7: NAMED ENTITY RECOGNITION
------------------------------


config.json:   0%|          | 0.00/998 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

Device set to use cpu


Text: Apple Inc. was founded by Steve Jobs in Cupertino, California. The company is now led by Tim Cook.

Named Entities:
Entity: 'Apple Inc' | Type: ORG | Confidence: 1.000
Entity: 'Steve Jobs' | Type: PER | Confidence: 0.993
Entity: 'Cupertino' | Type: LOC | Confidence: 0.977
Entity: 'California' | Type: LOC | Confidence: 0.999
Entity: 'Tim Cook' | Type: PER | Confidence: 1.000

Reflection: NER successfully identifies people, organizations, and locations with high accuracy.


In [10]:
# =============================================================================
# TASK 8: PEEK AT TOKENIZATION
# =============================================================================
print("TASK 8: TOKENIZATION ANALYSIS")
print("-" * 30)

# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Test text
test_text = "Hello, this is a tokenization example!"

print(f"Original text: {test_text}")
print()

# Tokenize
tokens = tokenizer.tokenize(test_text)
token_ids = tokenizer.encode(test_text)

print(f"Tokens: {tokens}")
print(f"Token IDs: {token_ids}")
print(f"Number of tokens: {len(tokens)}")
print()

# Show token-by-token breakdown
print("Token-by-token breakdown:")
for i, (token, token_id) in enumerate(zip(tokens, token_ids[1:-1])):  # Skip [CLS] and [SEP]
    print(f"  {i+1}. '{token}' -> ID: {token_id}")

print("\nReflection: Tokenization reveals how text is split into subwords, essential for model input processing.")
print("="*60)


TASK 8: TOKENIZATION ANALYSIS
------------------------------
Original text: Hello, this is a tokenization example!

Tokens: ['hello', ',', 'this', 'is', 'a', 'token', '##ization', 'example', '!']
Token IDs: [101, 7592, 1010, 2023, 2003, 1037, 19204, 3989, 2742, 999, 102]
Number of tokens: 9

Token-by-token breakdown:
  1. 'hello' -> ID: 7592
  2. ',' -> ID: 1010
  3. 'this' -> ID: 2023
  4. 'is' -> ID: 2003
  5. 'a' -> ID: 1037
  6. 'token' -> ID: 19204
  7. '##ization' -> ID: 3989
  8. 'example' -> ID: 2742
  9. '!' -> ID: 999

Reflection: Tokenization reveals how text is split into subwords, essential for model input processing.


In [11]:
# TASK 9: SENTENCE SIMILARITY WITH EMBEDDINGS
# =============================================================================
print("TASK 9: SENTENCE SIMILARITY")
print("-" * 30)

# Initialize model and tokenizer for embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Using alternative since sentence-transformers might not be available
from transformers import AutoModel, AutoTokenizer
import torch

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def get_sentence_embedding(sentence):
    inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # Use mean pooling of token embeddings
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.numpy()

# Test sentences
sentences = [
    "I love cats and dogs.",
    "I adore pets like cats and dogs.",
    "The weather is sunny today.",
    "Cats and dogs are my favorite animals."
]

print("Computing sentence similarities:")
embeddings = [get_sentence_embedding(sent) for sent in sentences]

print("\nSentence pairs and similarities:")
for i in range(len(sentences)):
    for j in range(i+1, len(sentences)):
        similarity = cosine_similarity(embeddings[i], embeddings[j])[0][0]
        print(f"Sentence {i+1} vs {j+1}: {similarity:.3f}")
        print(f"  '{sentences[i]}'")
        print(f"  '{sentences[j]}'")
        print()

print("Reflection: Embeddings capture semantic similarity, showing higher scores for related sentences.")
print("="*60)


TASK 9: SENTENCE SIMILARITY
------------------------------
Computing sentence similarities:

Sentence pairs and similarities:
Sentence 1 vs 2: 0.917
  'I love cats and dogs.'
  'I adore pets like cats and dogs.'

Sentence 1 vs 3: 0.644
  'I love cats and dogs.'
  'The weather is sunny today.'

Sentence 1 vs 4: 0.873
  'I love cats and dogs.'
  'Cats and dogs are my favorite animals.'

Sentence 2 vs 3: 0.637
  'I adore pets like cats and dogs.'
  'The weather is sunny today.'

Sentence 2 vs 4: 0.893
  'I adore pets like cats and dogs.'
  'Cats and dogs are my favorite animals.'

Sentence 3 vs 4: 0.660
  'The weather is sunny today.'
  'Cats and dogs are my favorite animals.'

Reflection: Embeddings capture semantic similarity, showing higher scores for related sentences.


In [15]:
# =============================================================================
# TASK 10: TINY FINE-TUNE
# =============================================================================
print("TASK 10: TINY FINE-TUNE")
print("-" * 30)

# Create a small toy dataset
toy_data = {
    "text": [
        "This movie is fantastic and amazing!",
        "I hate this terrible film.",
        "The movie was okay, nothing special.",
        "Absolutely loved this brilliant movie!",
        "Worst movie I've ever seen.",
        "It's an average film with decent acting.",
        "Outstanding performance and great story!",
        "Boring and poorly made movie.",
        "The film has some good moments.",
        "Incredible cinematography and acting!"
    ],
    "label": [1, 0, 1, 1, 0, 1, 1, 0, 1, 1]  # 1: positive/neutral, 0: negative
}

# Convert to datasets format
dataset = Dataset.from_dict(toy_data)
dataset = dataset.train_test_split(test_size=0.3, seed=42)

# Initialize model and tokenizer
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

# Tokenize dataset
def tokenize_function(examples):
    return tokenizer(examples["text"], truncation=True, padding=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir="./test_trainer",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="no",
    learning_rate=5e-5,
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Define metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": accuracy_score(labels, predictions)}

# Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print("Starting fine-tuning...")
trainer.train()

print("\nEvaluating model...")
eval_results = trainer.evaluate()
print("Evaluation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value}")

# Test on some examples and find misclassifications
test_texts = tokenized_dataset["test"]["text"]
test_labels = tokenized_dataset["test"]["label"]

predictions = trainer.predict(tokenized_dataset["test"])
predicted_labels = np.argmax(predictions.predictions, axis=1)

print(f"\nFinal Accuracy: {accuracy_score(test_labels, predicted_labels):.3f}")

print("\nAnalyzing predictions:")
misclassifications = []
for i, (text, true_label, pred_label) in enumerate(zip(test_texts, test_labels, predicted_labels)):
    status = "✓" if true_label == pred_label else "✗"
    print(f"{status} '{text}' | True: {true_label}, Pred: {pred_label}")
    if true_label != pred_label:
        misclassifications.append((text, true_label, pred_label))

if misclassifications:
    print(f"\nMisclassification Analysis:")
    text, true_label, pred_label = misclassifications[0]
    print(f"Example: '{text}'")
    print(f"The model predicted {pred_label} but the true label was {true_label}.")
    print("This could be due to the limited training data or ambiguous sentiment in the text.")
else:
    print("\nNo misclassifications found on this small test set!")

print("\nReflection: Fine-tuning achieved good performance even with minimal data and 1 epoch training.")
print("="*60)

print("ALL TASKS COMPLETED!")
print("Summary of what we accomplished:")
print("1. ✓ Sentiment analysis with pretrained model")
print("2. ✓ Zero-shot topic classification")
print("3. ✓ English-Hindi bidirectional translation")
print("4. ✓ Text summarization")
print("5. ✓ Extractive question answering")
print("6. ✓ Masked language modeling with BERT")
print("7. ✓ Named entity recognition")
print("8. ✓ Tokenization analysis")
print("9. ✓ Sentence similarity with embeddings")
print("10. ✓ Fine-tuning with accuracy evaluation")

TASK 10: TINY FINE-TUNE
------------------------------


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

Starting fine-tuning...


...


Evaluating model...


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Evaluation Results:
  eval_loss: 0.4377983510494232
  eval_accuracy: 1.0
  eval_runtime: 0.0988
  eval_samples_per_second: 30.353
  eval_steps_per_second: 10.118
  epoch: 5.0


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



Final Accuracy: 1.000

Analyzing predictions:
✓ 'It's an average film with decent acting.' | True: 1, Pred: 1
✓ 'Outstanding performance and great story!' | True: 1, Pred: 1
✓ 'This movie is fantastic and amazing!' | True: 1, Pred: 1

No misclassifications found on this small test set!

Reflection: Fine-tuning achieved good performance even with minimal data and 1 epoch training.
ALL TASKS COMPLETED!
Summary of what we accomplished:
1. ✓ Sentiment analysis with pretrained model
2. ✓ Zero-shot topic classification
3. ✓ English-Hindi bidirectional translation
4. ✓ Text summarization
5. ✓ Extractive question answering
6. ✓ Masked language modeling with BERT
7. ✓ Named entity recognition
8. ✓ Tokenization analysis
9. ✓ Sentence similarity with embeddings
10. ✓ Fine-tuning with accuracy evaluation


In [16]:
# Conclusion
# These 10 NLP tasks show how easy it's become to work with AI language models today.
# We went from simple things like checking if a review is positive or negative,
# all the way to training our own custom model - all with just a few lines of code!
#
# The cool part is that we didn't have to build these models from scratch;
# instead, we used pre-built ones that smart researchers already trained on millions of examples.
# It's like having a toolkit where each tool is already an expert at its job.
#
# This assignment proves that you don't need to be an AI researcher to use powerful language technology -
# you just need to know which tools to pick and how to use them effectively.


In [17]:
# Quick NLP Revision - Last Minute Study Guide
#
# Task 1: Sentiment Analysis
# What: Classify text as positive/negative/neutral
# Model: RoBERTa (Twitter-trained)
# Code: pipeline("sentiment-analysis")
# Key Point: No training needed, just load and predict
# ------------------------------------------------------------
#
# Task 2: Zero-Shot Classification
# What: Classify into ANY categories without training
# Model: BART + MNLI (Natural Language Inference)
# Code: pipeline("zero-shot-classification")
# Magic: Converts classification into "Does text match this label?" question
# ------------------------------------------------------------
#
# Task 3: Translation (EN ↔ HI)
# What: Convert English to Hindi and vice versa
# Model: Helsinki-NLP OPUS models
# Code: pipeline("translation")
# Remember: Need separate models for each direction
# ------------------------------------------------------------
#
# Task 4: Text Summarization
# What: Make long text shorter while keeping main points
# Model: T5-small (Text-to-Text Transfer Transformer)
# Code: pipeline("summarization")
# Parameters: max_length, min_length, do_sample=False
# ------------------------------------------------------------
#
# Task 5: Question Answering
# What: Find answers within a given passage
# Model: DistilBERT + SQuAD dataset
# Code: pipeline("question-answering")
# Input: Question + Context → Answer span + confidence
# ------------------------------------------------------------
#
# Task 6: Fill-in-the-Blank (BERT)
# What: Predict missing words using [MASK] token
# Model: BERT-base-uncased
# Code: pipeline("fill-mask")
# Core Idea: Uses bidirectional context (left + right words)
# ------------------------------------------------------------
#
# Task 7: Named Entity Recognition (NER)
# What: Find people, places, organizations in text
# Model: BERT-large-cased + CoNLL-2003
# Code: pipeline("ner")
# Types: PERSON, LOCATION, ORGANIZATION, MISCELLANEOUS
# ------------------------------------------------------------
#
# Task 8: Tokenization
# What: See how text breaks into pieces for AI models
# Model: BERT tokenizer (WordPiece algorithm)
# Code: tokenizer.tokenize() and tokenizer.encode()
# Why Important: Shows how AI "reads" your text
# ------------------------------------------------------------
#
# Task 9: Sentence Similarity
# What: Compare how similar two sentences are in meaning
# Method: Convert to vectors → calculate cosine similarity
# Code: Use BERT embeddings + cosine_similarity()
# Range: 0 (different) to 1 (identical meaning)
# ------------------------------------------------------------
#
# Task 10: Fine-tuning (Training)
# What: Teach a pretrained model your specific task
# Model: DistilBERT + custom data
# Process: Load model → Add your data → Train 1 epoch → Evaluate
# Key: Transfer learning - start with smart model, make it smarter for your task
# ------------------------------------------------------------
#
# Quick Model Cheat Sheet
#
# | Task            | Best Model Family | Why?                                |
# |-----------------|------------------|-------------------------------------|
# | Classification  | BERT/RoBERTa     | Great at understanding context      |
# | Generation      | T5/BART          | Built for text-to-text tasks        |
# | Translation     | Marian/OPUS      | Language-pair specialists           |
# | Embeddings      | BERT/Sentence-BERT | Rich semantic representations   |
# ------------------------------------------------------------
#
# Key Code Patterns
#
# Basic Pipeline Usage:
# classifier = pipeline("task-name", model="model-name")
# result = classifier("your text here")
#
# Fine-tuning Pattern:
# model = AutoModelForSequenceClassification.from_pretrained("model-name")
# trainer = Trainer(model=model, train_dataset=data)
# trainer.train()
# ------------------------------------------------------------
#
# Remember These Key Points
#
# 1. Pipelines = Easy Mode - Use for quick tasks, no setup needed
# 2. Pretrained = Smart Starting Point - Never train from scratch
# 3. Fine-tuning = Customization - Adapt general models to your specific needs
# 4. Tokenization = AI's Reading - Text → numbers that models understand
# 5. Embeddings = Meaning Vectors - Similar meanings → similar vectors
# 6. Transfer Learning = Efficiency - Leverage existing knowledge
# ------------------------------------------------------------
#
# Last-Minute Exam Tips
#
# If asked about model choice:
# - Small data/fast inference → DistilBERT
# - High accuracy needed → BERT-large/RoBERTa
# - Text generation → T5/BART
# - Multilingual → XLM-R
#
# If asked about when to fine-tune:
# - You have task-specific data
# - General models don't perform well enough
# - You need domain adaptation
#
# If asked about tokenization:
# - Subword splitting handles unknown words
# - Special tokens like [CLS], [SEP], [MASK]
# - Fixed vocabulary size (usually ~30K tokens)
#
# Common hyperparameters:
# - Learning rate: 5e-5 (fine-tuning), 1e-3 (training from scratch)
# - Batch size: 16-32 (small models), 4-8 (large models)
# - Epochs: 1-3 (fine-tuning), 10+ (from scratch)
